In [ ]:
import torch
import copy
from utils import evaluate_synset,get_daparam,get_network,get_dataset,ParamDiffAug
import os
import numpy as np
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["NCCL_P2P_DISABLE"]="1"

In [ ]:
syn_data = torch.load('syn_data/res_DC_CIFAR100_ConvNet_10ipc.pt') #res_DSA_MNIST_ConvNet_1ipc.pt
eval_it_pool = [1000] 
model_eval_pool = ['MLP', 'ConvNet', 'LeNet', 'AlexNet', 'VGG11', 'ResNet18']
model='ConvNet'
it=1000
dataset='CIFAR100'#'MNIST'
ipc=10
num_eval=5
channel=1
num_classes=100 #10
im_size=(32,32) #(28,28)

data_path='data'
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = get_dataset(dataset, data_path)

accs_all_exps = dict() # record performances of all experiments
for key in model_eval_pool:
    accs_all_exps[key] = []

In [ ]:
class arguments:
    pass
args = arguments()

In [ ]:
args.device='cuda' if torch.cuda.is_available() else 'cpu'
args.lr_net=0.01
args.epoch_eval_train=300
args.batch_train=256
args.dsa=True
args.dsa_strategy='color_crop_cutout_flip_scale_rotate' #'color_crop_cutout_scale_rotate'
args.Iteration=1000
args.dsa_param = ParamDiffAug()

In [ ]:
for syn in range(1):
    image_syn = syn_data['data'][syn][0]
    label_syn = syn_data['data'][syn][1]
    ''' Evaluate synthetic data '''
    for model_eval in model_eval_pool:
        print('-------------------------\nEvaluation\nmodel_train = %s, model_eval = %s, iteration = %d'%(model, model_eval, it))

        if args.dsa:
            epoch_eval_train = 1000
            args.dc_aug_param = None
            print('DSA augmentation strategy: \n', args.dsa_strategy)
            print('DSA augmentation parameters: \n', args.dsa_param.__dict__)
        else:
            args.dc_aug_param = get_daparam(dataset, model, model_eval, ipc) # This augmentation parameter set is only for DC method. It will be muted when args.dsa is True.
            print('DC augmentation parameters: \n', args.dc_aug_param)

        if args.dsa or args.dc_aug_param['strategy'] != 'none':
            epoch_eval_train = 1000  # Training with data augmentation needs more epochs.
        else:
            epoch_eval_train = 300

        accs = []
        for it_eval in range(num_eval):
            net_eval = get_network(model_eval, channel, num_classes, im_size).to(args.device) # get a random model
            image_syn_eval, label_syn_eval = copy.deepcopy(image_syn.detach()), copy.deepcopy(label_syn.detach()) # avoid any unaware modification
            _, acc_train, acc_test = evaluate_synset(it_eval, net_eval, image_syn_eval, label_syn_eval, testloader, args)
            accs.append(acc_test)
        print('Evaluate %d random %s, mean = %.4f std = %.4f\n-------------------------'%(len(accs), model_eval, np.mean(accs), np.std(accs)))

        if it == args.Iteration: # record the final results
            accs_all_exps[model_eval] += accs